In [1]:
import numpy as np
import cobra
import snek

In [2]:
model = cobra.io.read_sbml_model('iEC1356_Bl21DE3.xml')

In [3]:
model

Name,iEC1356_Bl21DE3
Memory address,7faeee42aeb0
Number of metabolites,1918
Number of reactions,2740
Number of genes,1356
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, periplasm, extracellular space"


In [4]:
snek.analysis.get_constrained_reactions(model)

,name,id,lower_bound,upper_bound
ATPM,ATP maintenance requirement,ATPM,3.15,1000.0
EX_cbl1_e,Cob(I)alamin exchange,EX_cbl1_e,-0.01,1000.0
EX_glc__D_e,D-Glucose exchange,EX_glc__D_e,-10.00,1000.0
CAT,Catalase,CAT,0.00,0.0
DHPTDNR,Dihydropteridine reductase,DHPTDNR,0.00,0.0
DHPTDNRN,Dihydropteridine reductase (NADH),DHPTDNRN,0.00,0.0
FHL,Formate-hydrogen lyase,FHL,0.00,0.0
SUCMALtpp,Succinate:malate antiporter (periplasm),SUCMALtpp,0.00,0.0
SUCTARTtpp,Succinate:D-tartrate antiporter (periplasm),SUCTARTtpp,0.00,0.0
SUCASPtpp,Succinate:aspartate antiporter (periplasm),SUCASPtpp,0.00,0.0


# Include Protein Synthesis Reaction in the Model

In [5]:
from collections import Counter
import pandas as pd

# sequence beta lactamase https://www.uniprot.org/uniprotkb/B5LY47/entry
# seq = 'AACIPLLLGSAPLYAQTSAVQQKLAALEKSSGGRLGVALIDTADNTQVLYRGDERFPMCSTSKVMAAAAVLKQSETQKQLLNQPVEIKPADLVNYNPIAEKHVNGTMTLAELSAAALQYSDNTAMNKLIAQLGGPGGVTAFARAIGDETFRLDRTEPTLNTAIPGDPRDTTTPRAMAQTLRQLTLGHALGETQRAQLVTWLKGNTTGAASIRAGLPTSWTVGDKTGSGGYGTTNDIAVIWPQGRAPLVLVTYFTQPQQNAESRRDVLASAARIIAEGL'
seq = "MASVENKEETPETPETDSEEEVTIKANLIFANGSTQTAEFKGTFEKATSEAYAYADTLKKDNGEYTVDVADKGYTLNIKFAGKEKTPEEPKEEVTIKANLIYADGKTQTAEFKGTFEEATAEAYRYADALKKDNGEYTVDVADKGYTLNIKFAGKEKTPEEPKEEVTIKANLIYADGKTQTAEFKGTFEEATAEAYRYADLLAKENGKYTVDVADKGYTLNIKFAGKEKTPEEPKEEVTIKANLIYADGKTQTAEFKGTFAEATAEAYRYADLLAKENGKYTADLEDGGYTINIRFAGKKVDEKPEEKEQVTIKENIYFEDGTVQTATFKGTFAEATAEAYRYADLLSKEHGKYTADLEDGGYTINIRFAGHHHHHH"
dic = Counter(seq)
# create a nice data frame
new = {}
for i, j in enumerate(dic.items()):
    new[i] = j
AAs = pd.DataFrame(new,index=['AA','#total']).T
print(AAs.shape)

(18, 2)


In [6]:
# Naming conventions for P. pastoris 
names = {'A': 'ala__L_c','C': 'cys__L_c','D': 'asp__L_c','E': 'glu__L_c','F': 'phe__L_c','G': 'gly_c',
             'H': 'his__L_c','I': 'ile__L_c','K': 'lys__L_c','L': 'leu__L_c','M': 'met__L_c','N': 'asn__L_c',
             'P': 'pro__L_c','Q': 'gln__L_c','R': 'arg__L_c','S': 'ser__L_c','T': 'thr__L_c','V': 'val__L_c',
             'W': 'trp__L_c','Y': 'tyr__L_c'} 

def calculate_stoichiometry(AAdf,protein_name='protein_c'):
    """
    Calculates the protein synthesis reaction stoichiometry.
    
    Parameters
    ----------
        AAdf   : pandas.DataFrame with at least 2 columns.
                 Column labeled "AA" contains the 1-letter code for all amino acids of the protein.
                 Column labeled "#total" contains the total number of the respective amino acid in the protein.
    Returns
    -------
        stoichiometry : dic
                        Stoichiometry that can be parsed to Cobra.core.reactions.Reaction.add_metabolites()
    """
    
    stoichiometry ={}
    # add all amino acids to reaction
    sum_aa = 0
    for i in AAdf.index:
        if AAdf.loc[i,'#total'] != 0:
            sum_aa += AAdf.loc[i,'#total']
            stoichiometry[names[AAdf.loc[i,'AA']]] = float(-AAdf.loc[i,'#total'])

    # add ATP consumption to reaction
    stoichiometry['atp_c'] = -sum_aa * 4.324 # [1]
    stoichiometry['h2o_c'] = -sum_aa * 4.324 # [1]
    stoichiometry['adp_c'] =  sum_aa * 4.324 # [1]
    stoichiometry['h_c'  ] =  sum_aa * 4.324 # [1]
    stoichiometry['pi_c' ] =  sum_aa * 4.324 # [1]
    # per peptide bond 1 H2O is produced
    stoichiometry['h2o_c'] += sum_aa -1
    
    # add protein metabolite to reaction
    stoichiometry[protein_name] = 1
    
    # [1] Cunningham, D. S., Koepsel, R. R., Ataai, M. M., & Domach, M. M. (2009). 
    # Factors affecting plasmid production in Escherichia coli from a resource 
    # allocation standpoint. Microbial Cell Factories, 8(1), 1-17.
    
    return stoichiometry

def include_protein(model,AAdf):
    '''
    Includes protein synthesis and sink reaktion in model.
    
    Parameters
    ----------
        AAdf   : pandas.DataFrame with at least 2 columns.
                 Column labeled "AA" contains the 1-letter code for all amino acids of the protein.
                 Column labeled "#total" contains the total number of the respective amino acid in the protein.
    '''
    
    # STEP 1: CREATE REACTION
    protein_synthesis = cobra.Reaction('protein_synthesis')
    protein_synthesis.name = "Synthesis of POI"
    protein_synthesis.subsystem = 'Protein Production'
    protein_synthesis.lower_bound = 0
    protein_synthesis.upper_bound = 1000
    model.add_reactions([protein_synthesis])
    
    # STEP 2: CREATE PROTEIN METABOLITE
    protein_c = cobra.Metabolite('protein_c')
    protein_c.formula = snek.elements.get_protein_formula(seq)
    protein_c.charge =  0
    protein_c.name = 'protein'
    protein_c.compartment = 'c'
    model.add_metabolites([protein_c])
    
    # STEP 3: Add STOICHIOMETRY to REACTION (makes use of 'protein_c' and has therefore to be step #3)
    model.reactions.protein_synthesis.add_metabolites(calculate_stoichiometry(AAdf))
    
    # STEP 4: Check MASS BALANCE
    balance = model.reactions.protein_synthesis.check_mass_balance()
    for element in balance:
        assert np.isclose(balance[element],0,atol=model.tolerance,rtol=0), f'Element "{element}" Not Mass Balanced: {balance[element]}'
    
    # STEP 5: ADD SINK REACTION
    model.add_boundary(model.metabolites.protein_c,type='sink',lb=0,ub=1000)
    
    return

In [7]:
with model:
    print(len(model.reactions),model.slim_optimize())
    include_protein(model,AAs)
    print(len(model.reactions),model.slim_optimize())
    model2 = model.copy()
model2.id   = 'iEC1356_Bl21DE3_protein'
model2.name = 'iEC1356_Bl21DE3 with protein synthesis'
cobra.io.write_sbml_model(model2,'iEC1356_Bl21DE3_protein.xml')

2740 0.9767013604875502
2742 0.9767013604875532


# Calculate Yields

In [8]:
model = cobra.io.read_sbml_model('iEC1356_Bl21DE3_protein.xml')

# Y_(X/gly), Y_(X/O2)

In [9]:
with model as tmp:
    # remove all C-influxes
    snek.set_bounds(tmp,'EX_glc__D_e',lb=0,ub=1000)
    snek.set_bounds(tmp,'EX_cbl1_e',lb=0,ub=1000)
    # remove ATPM
    snek.set_bounds(tmp,'ATPM',lb=0,ub=1000)
    # set objective
    snek.set_objective(tmp,'BIOMASS_Ec_iJO1366_core_53p95M','max')
    # add glycerol influx
    snek.set_bounds(tmp,'EX_glyc_e',lb=-10,ub=0)
    # maximize and fix max value
    max_ = tmp.slim_optimize()
    snek.set_bounds(tmp,'BIOMASS_Ec_iJO1366_core_53p95M',lb=max_,ub=max_)
    # set objective: min O2 consumption
    snek.set_objective(tmp,'EX_o2_e','max')
    # calculate pFBA
    solution = snek.sensitive_optimize(model,pFBA=True)

In [10]:
gly_rate = - solution['EX_glyc_e'] * model.metabolites.glyc_e.formula_weight / 1000 # g/(g h)
o2_rate  = - solution['EX_o2_e']   * model.metabolites.o2_e.formula_weight   / 1000 # g/(g h)
bm_rate  = solution['BIOMASS_Ec_iJO1366_core_53p95M'] # g/(g h) = 1/h

print(f'Y_(X/gly) = {bm_rate/gly_rate:6.3f} g/g')
print(f'Y_(X/O2)  = {bm_rate/o2_rate:6.3f} g/g')

Y_(X/gly) =  0.627 g/g
Y_(X/O2)  =  1.788 g/g


# Y_(ATP/gly), Y_(ATP/O2)

In [11]:
with model as tmp:
    # remove all C-influxes
    snek.set_bounds(tmp,'EX_glc__D_e',lb=0,ub=1000)
    snek.set_bounds(tmp,'EX_cbl1_e',lb=0,ub=1000)
    # remove ATPM
    snek.set_bounds(tmp,'ATPM',lb=0,ub=1000)
    # set objective
    snek.set_objective(tmp,'ATPM','max')
    # add glycerol influx
    snek.set_bounds(tmp,'EX_glyc_e',lb=-10,ub=0)
    # maximize and fix max value
    max_ = tmp.slim_optimize()
    snek.set_bounds(tmp,'ATPM',lb=max_,ub=max_)
    # set objective: min O2 consumption
    snek.set_objective(tmp,'EX_o2_e','max')
    # calculate pFBA
    solution = snek.sensitive_optimize(model,pFBA=True)

In [12]:
gly_rate = - solution['EX_glyc_e'] * model.metabolites.glyc_e.formula_weight / 1000 # g/(g h)
o2_rate  = - solution['EX_o2_e']   * model.metabolites.o2_e.formula_weight   / 1000 # g/(g h)
bm_rate  =   solution['ATPM']      * model.metabolites.atp_c.formula_weight  / 1000 # g/(g h)

print(f'Y_(ATP/gly) = {bm_rate/gly_rate:6.3f} g/g')
print(f'Y_(ATP/O2)  = {bm_rate/o2_rate:6.3f} g/g')

Y_(ATP/gly) = 73.756 g/g
Y_(ATP/O2)  = 60.650 g/g


# Y_(P/gly), Y_(P/O2)

In [13]:
with model as tmp:
    # remove all C-influxes
    snek.set_bounds(tmp,'EX_glc__D_e',lb=0,ub=1000)
    snek.set_bounds(tmp,'EX_cbl1_e',lb=0,ub=1000)
    # remove ATPM
    snek.set_bounds(tmp,'ATPM',lb=0,ub=1000)
    # set objective
    snek.set_objective(tmp,'protein_synthesis','max')
    # add glycerol influx
    snek.set_bounds(tmp,'EX_glyc_e',lb=-10,ub=0)
    # maximize and fix max value
    max_ = tmp.slim_optimize()
    snek.set_bounds(tmp,'protein_synthesis',lb=max_,ub=max_)
    # set objective: min O2 consumption
    snek.set_objective(tmp,'EX_o2_e','max')
    # calculate pFBA
    solution = snek.sensitive_optimize(model,pFBA=True)

In [14]:
gly_rate = - solution['EX_glyc_e'] * model.metabolites.glyc_e.formula_weight / 1000 # g/(g h)
o2_rate  = - solution['EX_o2_e']   * model.metabolites.o2_e.formula_weight   / 1000 # g/(g h)
bm_rate  =   solution['protein_synthesis']      * model.metabolites.protein_c.formula_weight  / 1000 # g/(g h)

print(f'Y_(P/gly) = {bm_rate/gly_rate:6.3f} g/g')
print(f'Y_(P/O2)  = {bm_rate/o2_rate:6.3f} g/g')

Y_(P/gly) =  0.652 g/g
Y_(P/O2)  =  2.501 g/g
